# DESCRIPTION

*   @Author:  Brayan Alberto Castellanso Bernal 
*   @Date: 2022/03/18 
*   Description: In this notebook we are going to apply the algorithm on a data set of Tesla shares on the United States stock market, where we will find:

    *   The CLOSE signal (daily closing price)
    *   The VOLUME signal (number of shares that have changed hands over that period of time).







#DATOS DE MERCADO DE VALORES DE MICROSOFT

Importando las librerias 

In [30]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import preprocessing  #para normalizar los datos antes de aplicar al algoritmo
from sklearn.metrics import pairwise_distances_argmin_min

import matplotlib.pyplot as plt
import seaborn as sb
from mpl_toolkits.mplot3d import Axes3D

#configuraciones basicas para PLT
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

#Descargando data usando Web-scrawping

In [41]:
#UrlWeb="https://es.wikipedia.org/wiki/Departamentos_de_Colombia"
#ClassName="wikitable sortable jquery-tablesorter"
UrlWeb= "https://finance.yahoo.com/quote/MSFT/history?period1=511056000&period2=1648166400&interval=1wk&filter=history&frequency=1wk&includeAdjustedClose=true"
ClassName="W(100%) M(0)"

In [42]:
import requests                           # libreria para manejar pedidos internet
from bs4 import BeautifulSoup             # libreria para parsear documentos HTML

In [43]:
!pip install kora -q
!apt install chromium-chromedriver
!pip install selenium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (99.0.4844.51-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [44]:
from kora.selenium import wd
wd.get(UrlWeb)
soup = BeautifulSoup(wd.page_source, 'html.parser')

In [45]:
tabla= soup.find_all(class_=ClassName)
df_list_table=pd.read_html(str(tabla))                                      # LEEMOS el THEAD y TBODY y lo guardamos en una lista
df=pd.DataFrame(df_list_table[0])

In [46]:
#df.head(10)                                                       #LAS PRIMERAS FILAS
df.tail(10)

,Date,Open,High,Low,Close*,Adj Close**,Volume
91,"Aug 10, 2020",211.67,211.88,203.14,208.90,205.70,141723900
92,"Aug 03, 2020",211.52,217.64,210.31,212.48,209.22,217546700
93,"Jul 27, 2020",201.47,205.10,199.01,205.01,201.87,149168500
94,"Jul 20, 2020",205.00,213.94,197.51,201.30,198.22,231764900
95,"Jul 13, 2020",214.48,215.80,201.39,202.88,199.77,169482800
96,"Jul 06, 2020",208.83,216.38,207.99,213.67,210.40,158397600
97,"Jun 29, 2020",195.78,208.02,193.55,206.26,203.10,122388900
98,"Jun 22, 2020",195.79,203.95,194.88,196.33,193.32,182956600
99,"Jun 15, 2020",184.58,199.29,184.01,195.15,192.16,168517400
100,*Close price adjusted for splits.**Adjusted cl...,*Close price adjusted for splits.**Adjusted cl...,*Close price adjusted for splits.**Adjusted cl...,*Close price adjusted for splits.**Adjusted cl...,*Close price adjusted for splits.**Adjusted cl...,*Close price adjusted for splits.**Adjusted cl...,*Close price adjusted for splits.**Adjusted cl...


In [47]:
#tenemosque borrar la ultima fila
df = df[:-1]

#Conociendo los datos descargados

In [48]:
print(df.info())
print("____________describe____________\n",df.describe())
print("____________nulos?____________\n",df.isnull().sum())
#print("Rows, Cols->",df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         100 non-null    object
 1   Open         100 non-null    object
 2   High         100 non-null    object
 3   Low          100 non-null    object
 4   Close*       100 non-null    object
 5   Adj Close**  100 non-null    object
 6   Volume       100 non-null    object
dtypes: object(7)
memory usage: 5.6+ KB
None
____________describe____________
                 Date           Open           High            Low  \
count            100            100            100            100   
unique           100             96             96             96   
top     Mar 21, 2022  0.56 Dividend  0.56 Dividend  0.56 Dividend   
freq               1              4              4              4   

               Close*    Adj Close**         Volume  
count             100            100            1

#Normalización de los datos.

##Convertimos los campos object a datetime o numeric (int/float)

In [49]:
df['Date']= df['Date'].str.replace(' ene ','/01/')
df['Date']= df['Date'].str.replace(' feb ','/02/')
df['Date']= df['Date'].str.replace(' mar ','/03/')
df['Date']= df['Date'].str.replace(' abr ','/04/')
df['Date']= df['Date'].str.replace(' may ','/05/')
df['Date']= df['Date'].str.replace(' jun ','/06/')
df['Date']= df['Date'].str.replace(' jul ','/07/')
df['Date']= df['Date'].str.replace(' ago ','/08/')
df['Date']= df['Date'].str.replace(' sep ','/09/')
df['Date']= df['Date'].str.replace(' oct ','/10/')
df['Date']= df['Date'].str.replace(' nov ','/11/')
df['Date']= df['Date'].str.replace(' dic ','/12/')

In [ ]:
df['Date']=pd.to_datetime(df['Date'], dayfirst = True)        # Convertimos el campo FECHA en tipo DATETIME
df['Open'] = pd.to_numeric(df['Open'])                        #  convertimos el campo a numerico 
df['High'] = pd.to_numeric(df['High'])
df['Low'] = pd.to_numeric(df['Low'])
df['Close*'] = pd.to_numeric(df['Close*'])
df['Adj Close**'] = pd.to_numeric(df['Adj Close**'])
df['Volume'] = pd.to_numeric(df['Volume'])

#Eliminando columnas que no necesitaremos

In [51]:
#borrar nulos o no?
df = df.dropna()
df = df.reset_index(drop=True)

In [52]:
# Eliminamos columna de fecha que no usaremos en el algoritmo.
dates = df['Date'] # Guardamos la columna Date.
df = df.drop('Date', 1) # Borramos la columna del dataframe.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


Normaliza todos los datos entre [0, 1].

In [ ]:
#%% Se normalizan los datos con MinMax()
min_max_scaler = preprocessing.MinMaxScaler() 
df_escalado = min_max_scaler.fit_transform(df)
df_escalado = pd.DataFrame(df_escalado) # Hay que convertir a DF el resultado.
df_escalado = df_escalado.rename(columns = {0:'Abrir',1:'Alto',2:'Bajo',3:'Cierre*',4: 'Cierre ajustado**', 5: 'Volumen'})

In [ ]:
df_escalado.head()
#df_escalado.isnull().sum()

#Representación gráfica de los datos.

In [ ]:
#%% Representación gráfica de los datos.
x = df_escalado['Close*'].values
y = df_escalado['Volume'].values
plt.xlabel('Cierre de precios')
plt.ylabel('Volumen')
plt.title('Microsoft CIERRE vs. VOLUMEN')
plt.plot(x,y,'o',markersize=3)

#Aplicación de k-means.

# Realizar una gráfica elbow o de codo

In [ ]:
#%% Curva elbow para determinar valor óptimo de k.
nc = range(1, 30) # El número de iteraciones que queremos hacer.
kmeans = [KMeans(n_clusters=i) for i in nc]
score = [kmeans[i].fit(df_escalado).score(df_escalado) for i in range(len(kmeans))]
score
plt.xlabel('Número de clústeres (k)')
plt.ylabel('Suma de los errores cuadráticos')
plt.plot(nc,score)

#Aplicar el algoritmo de k-mean

In [ ]:
#%% Aplicación de k-means con k = 5.
kmeans = KMeans(n_clusters=5).fit(df_escalado)
centroids = kmeans.cluster_centers_
print(centroids)   #coordenadas de los centroides

#Etiquetado de datos.

In [ ]:
#%% Etiquetamos nuestro dataframe.
labels = kmeans.predict(df_escalado)   #generamos las etiquetas de cada clouster
df['label'] = labels                   #añadimos la columna “label” a nuestro dataframe original

df.insert(0, 'Date', dates)           #%% Añadimos la columna de fecha

In [ ]:
df.head(10)

#Representación gráfica de los clústeres k-means

In [ ]:
#%% Plot k-means clustering.
colores=['red','green','blue','yellow','fuchsia']
asignar=[]
for row in labels:
     asignar.append(colores[row])
plt.scatter(x, y, c=asignar, s=4)
plt.scatter(centroids[:, 0], centroids[:, 1], marker='*', c='black', s=20) # Marco centroides.
plt.xlabel('Precio de cierre')
plt.ylabel('Volumen')
plt.title('Tesla stocks k-means clustering')
plt.show()